In [2]:
# 코어 스파크 라이브러리를 임포트
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = (
    SparkSession
    .builder
    .appName("Book Pipeline Project")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정
from IPython.display import display, display_pretty, clear_output, JSON
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size
spark

22/06/07 06:12:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Data Loading

In [3]:
book = spark.read.parquet("book_table/total/*.parquet")
book.printSchema()
# book.show(truncate=False)
display(book)

root
 |-- author: string (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- date: long (nullable = true)
 |-- description: string (nullable = true)
 |-- image: string (nullable = true)
 |-- ranking: integer (nullable = true)
 |-- title: string (nullable = true)



author,book_id,category,date,description,image,ranking,title
토마스 불핀치,2855,신화,1648447114000,중학생 독후감 필독선 시리즈 제13권 토마스 불핀치의 명작 그리스 로마 신화를 작품 알고 들어가기 내용 훑어보기 작품 분석하기 작가 들여다보기 시대와 연관짓기 작품 토론하기 ...,https://bookthumb-phinf.pstatic.net/cover/000/028/00002855.jpg?type=m5,34,중학생이 보는 그리스 로마 신화
효성출판사 편집부,40565,가구/DIY,1648449401000,분위기기능에 따른 가구선택과 배치 주문가구 정보전국 5대 가구시장을 컬러화보 위주로 엮은 인테리어 무크지,https://bookthumb-phinf.pstatic.net/cover/000/405/00040565.jpg?type=m5,37,소가구 & 수납가구
중앙M&B 편집부,66769,가구/DIY,1648449401000,실내 벽에서 소품가구까지 활용 가능한 페인팅 기법 을 컬러사진과 함께 설명한 생활무크 페인팅에 필요 한 도구와 재료설명을 비롯해 색다르게 칠하는 페인팅기법리폼 및 장식 아이디...,https://bookthumb-phinf.pstatic.net/cover/000/667/00066769.jpg?type=m5,36,내 손으로 더 예쁘게 페인팅 DIY
제레미 리프킨,118837,경제/경영,null,산업 시대 초기에 석탄과 증기 기관이 새로운 경제 패러다임을 마련했듯이 이제 수소 에너지가 기존의 경제 정치 사회를 근본적으로 바꿀 것이라고 예견하는 경제서 노동의 종말 소유...,https://bookthumb-phinf.pstatic.net/cover/001/188/00118837.jpg?type=m5,27,수소 혁명
편집부,149663,비즈/구슬공예,1648449269000,스와롭스키 비즈로 귀엽고 깜찍한 동물들을 만든다 꿰는 방법을 쉽게 이해하도록 평면도로 그려 놓았으며 완성된 크기와 사용할 비즈 낚싯줄 길이 공정도 꿰는 방법 등을 세세하게 기...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149663.jpg?type=m5,39,비즈 캐릭터
노경민,149664,비즈/구슬공예,1648449268000,비즈 공예 입문서이자 전문서 PART 1에서는 시작 마무리 규칙적인 기법 비즈의 기본적인 종류와 명칭 간단한 도구들에 대해 알아보며 PART 2에서는 진주 PART 3에서는 ...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149664.jpg?type=m5,38,비즈 주얼리
편집부,150897,비즈/구슬공예,1648449270000,비즈 만드는 법 303가지를 모아 엮었다 반지 목걸이 팔찌 귀걸이 등 다양한 악세사리 제작 방법과 비즈의 종류 등에 대하여 일목요연하게 기술하고 있으며 부재료 사용법과 게재작...,https://bookthumb-phinf.pstatic.net/cover/001/508/00150897.jpg?type=m5,40,토탈 비즈 컬렉션
사메지마 다카코,150950,비즈/구슬공예,1648449268000,세상에서 단 하나뿐인 나만의 보석 만들기 로맨틱한 디자인부터 귀엽고 깜찍한 디자인까지 반지 초커 피어스 목걸이 핸드폰 줄 헤어 핀 등을 이용하여 사랑스럽고 기품있는 비즈 쥬얼...,https://bookthumb-phinf.pstatic.net/cover/001/509/00150950.jpg?type=m5,37,프린세스 비즈
서울문화사 편집부,155380,비즈/구슬공예,1648449267000,3천원으로 제작하는 명품 스타일 비즈 액세서리 이 책은 비즈로 팔찌 발찌 반지 브로치 등의 액세서리를 포함해서 핸드폰줄과 키홀더 등도 제작할 수 있도록 구성되었다 종류별로 구...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155380.jpg?type=m5,35,3천원으로 명품 스타일 비즈 액세서리
서울문화사 편집부,155381,비즈/구슬공예,1648449267000,탄생석 목걸이 · 귀고리 96가지 이 책은 비즈로 목걸이와 귀고리를 제작할 수 있도록 구성되었다 목걸이 귀고리 목걸이 귀고리 세트로 구별해 놓아 찾기 쉽게 해 놓았으며 작품...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155381.jpg?type=m5,36,행운 부르는 비즈


In [4]:
book.createOrReplaceTempView("book")
spark.sql("show tables 'book'")

database,tableName,isTemporary
,book,true


In [5]:
spark.sql("select count(*) as count from book")

count
4791


In [6]:
spark.sql("select category, count(*) as count from book group by category")

category,count
쉽게 읽는 과학,11
집/살림,39
천문/지구과학,32
한국사,35
불교,37
서양사,35
독서 에세이,37
연애/사랑 에세이,40
취미/레저,35
인간관계,23


## Data PreProcessing

In [4]:
!pip install konlpy # notebook 이미지 수정하기

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt

#### 조사, 어미, 어간 등 제거

In [6]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

colsClean = udf(lambda z : clean(z), ArrayType(StringType()))
spark.udf.register("colsClean", colsClean)

def clean(s):
    okt = Okt()
    clean_words = []
    for word in okt.pos(s, stem=True):
#         if word[1] in ['Noun', 'Verb', 'Adjective']:
        if word[1] in ['Noun', 'Adjective']:
            clean_words.append(word[0])
    new_doc = ' '.join(clean_words)
    return clean_words
            
tmp = book.withColumn('preprocessed', colsClean('description'))

In [7]:
tmp

author,book_id,category,date,description,image,ranking,title,preprocessed
토마스 불핀치,2855,신화,1648447114000,중학생 독후감 필독선 시리즈 제13권 토마스 불핀치의 명작 그리스 로마 신화를 작품 알고 들어가기 내용 훑어보기 작품 분석하기 작가 들여다보기 시대와 연관짓기 작품 토론하기 ...,https://bookthumb-phinf.pstatic.net/cover/000/028/00002855.jpg?type=m5,34,중학생이 보는 그리스 로마 신화,"[중학생, 독후감, 필, 독선, 시리즈, 제, 권, 토마스, 불, 핀치, 명작, 그리스, 로마, 신화, 작품, 알, 내용, 훑어보기, 작품, 분석, 작가, 시대, 연관, 작품..."
효성출판사 편집부,40565,가구/DIY,1648449401000,분위기기능에 따른 가구선택과 배치 주문가구 정보전국 5대 가구시장을 컬러화보 위주로 엮은 인테리어 무크지,https://bookthumb-phinf.pstatic.net/cover/000/405/00040565.jpg?type=m5,37,소가구 & 수납가구,"[분위기, 기능, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 가구, 시장, 컬러, 화보, 위주, 엮, 인테리어, 무크]"
중앙M&B 편집부,66769,가구/DIY,1648449401000,실내 벽에서 소품가구까지 활용 가능한 페인팅 기법 을 컬러사진과 함께 설명한 생활무크 페인팅에 필요 한 도구와 재료설명을 비롯해 색다르게 칠하는 페인팅기법리폼 및 장식 아이디...,https://bookthumb-phinf.pstatic.net/cover/000/667/00066769.jpg?type=m5,36,내 손으로 더 예쁘게 페인팅 DIY,"[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설명, 생활, 무크, 페인팅, 필요, 도구, 재료, 설명, 비롯, 색다르다, 칠하다, 페인, ..."
제레미 리프킨,118837,경제/경영,null,산업 시대 초기에 석탄과 증기 기관이 새로운 경제 패러다임을 마련했듯이 이제 수소 에너지가 기존의 경제 정치 사회를 근본적으로 바꿀 것이라고 예견하는 경제서 노동의 종말 소유...,https://bookthumb-phinf.pstatic.net/cover/001/188/00118837.jpg?type=m5,27,수소 혁명,"[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 이제, 수소, 에너지, 기존, 경제, 정치, 사회, 근본, 것, 예견, 경제, 노동, 종말, 소..."
편집부,149663,비즈/구슬공예,1648449269000,스와롭스키 비즈로 귀엽고 깜찍한 동물들을 만든다 꿰는 방법을 쉽게 이해하도록 평면도로 그려 놓았으며 완성된 크기와 사용할 비즈 낚싯줄 길이 공정도 꿰는 방법 등을 세세하게 기...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149663.jpg?type=m5,39,비즈 캐릭터,"[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 꿰, 방법, 이해, 평면도, 완성, 크기, 사용, 비즈, 낚싯줄, 길이, 공정, 꿰, 방법, 등, 기록, 누구, 작품, ..."
노경민,149664,비즈/구슬공예,1648449268000,비즈 공예 입문서이자 전문서 PART 1에서는 시작 마무리 규칙적인 기법 비즈의 기본적인 종류와 명칭 간단한 도구들에 대해 알아보며 PART 2에서는 진주 PART 3에서는 ...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149664.jpg?type=m5,38,비즈 주얼리,"[비즈, 공예, 입문, 이자, 전문, 시작, 마무리, 규칙, 기법, 비즈, 기본, 종류, 명칭, 간단하다, 도구, 대해, 진주, 크리스털, 이용, 기본, 기법, 비즈, 공예,..."
편집부,150897,비즈/구슬공예,1648449270000,비즈 만드는 법 303가지를 모아 엮었다 반지 목걸이 팔찌 귀걸이 등 다양한 악세사리 제작 방법과 비즈의 종류 등에 대하여 일목요연하게 기술하고 있으며 부재료 사용법과 게재작...,https://bookthumb-phinf.pstatic.net/cover/001/508/00150897.jpg?type=m5,40,토탈 비즈 컬렉션,"[비즈, 법, 가지, 엮었다, 반지, 목걸이, 팔찌, 귀걸이, 등, 다양하다, 악세사리, 제작, 방법, 비즈, 종류, 등, 대하, 일목요연하다, 기술, 있다, 부, 재료, 용..."
사메지마 다카코,150950,비즈/구슬공예,1648449268000,세상에서 단 하나뿐인 나만의 보석 만들기 로맨틱한 디자인부터 귀엽고 깜찍한 디자인까지 반지 초커 피어스 목걸이 핸드폰 줄 헤어 핀 등을 이용하여 사랑스럽고 기품있는 비즈 쥬얼...,https://bookthumb-phinf.pstatic.net/cover/001/509/00150950.jpg?type=m5,37,프린세스 비즈,"[세상, 단, 하나, 나, 보석, 만들기, 로맨틱, 디자인, 귀엽다, 깜찍하다, 디자인, 반지, 초, 피어스, 목걸이, 핸드폰, 줄, 헤어, 핀, 등, 이용, 사랑스럽다, 기..."
서울문화사 편집부,155380,비즈/구슬공예,1648449267000,3천원으로 제작하는 명품 스타일 비즈 액세서리 이 책은 비즈로 팔찌 발찌 반지 브로치 등의 액세서리를 포함해서 핸드폰줄과 키홀더 등도 제작할 수 있도록 구성되었다 종류별로 구...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155380.jpg?type=m5,35,3천원으로 명품 스타일 비즈 액세서리,"[제작, 명품, 스타일, 비즈, 액세서리, 이, 책, 비즈, 팔찌, 발찌, 반지, 브로치, 등, 액세서리, 포함, 핸드폰, 줄, 키홀, 더, 등, 제작, 수, 있다, 구성, ..."
서울문화사 편집부,155381,비즈/구슬공예,1648449267000,탄생석 목걸이 · 귀고리 96가지 이 책은 비즈로 목걸이와 귀고리를 제작할 수 있도록 구성되었다 목걸이 귀고리 목걸이 귀고리 세트로 구별해 놓아 찾기 쉽게 해 놓았으며 작품...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155381.jpg?type=m5,36,행운 부르는 비즈,"[탄생석, 목걸이, 귀고리, 가지, 이, 책, 비즈, 목걸이, 귀고리, 제작, 수, 있다, 구성, 목걸이, 귀고리, 목걸이, 귀고리, 세트, 구별, 찾기, 해, 작품, 난이도..."


#### 불용어 제거

In [11]:
# import numpy
# import pandas as pd
# import nltk
# df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
# df[0] = df[0].apply(lambda x: x.strip())
# stopwords = df[0].to_numpy()

In [12]:
# stopwords

In [8]:
# stopwords = spark.read.text("stopwords_kor.txt")
# stopwords.selectExpr("split(value, ' ')")
import numpy
with open("stopwords_kor.txt", 'r') as f:
    stopwords = f.readlines()
stopwords = str(stopwords[0]).split(" ")
stopwords = numpy.array(stopwords)
stopwords

array(['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해',
       '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여',
       '입각하여', '기준으로', '예하면', '예를', '들면', '예를', '들자면', '저', '소인', '소생',
       '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수', '없다',
       '해서는', '안된다', '뿐만', '아니라', '만이', '아니다', '만은', '아니다', '막론하고',
       '관계없이', '그치지', '않다', '그러나', '그런데', '하지만', '든간에', '논하지', '않다',
       '따지지', '않다', '설사', '비록', '더라도', '아니면', '만', '못하다', '하는', '편이',
       '낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다',
       '오르다', '제외하고', '이', '외에', '이', '밖에', '하여야', '비로소', '한다면', '몰라도',
       '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할', '생각이다', '하려고하다',
       '이리하여', '그리하여', '그렇게', '함으로써', '하지만', '일때', '할때', '앞에서', '중에서',
       '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다',
       '할수있어', '임에', '틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만',
       '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼',
      

In [9]:
# nltk.download('punkt')

stopCols = udf(lambda x : cleanSW(x), StringType())
spark.udf.register("stopCols", stopCols)
def cleanSW(s):
    clean_words = []
#     for word in nltk.tokenize.word_tokenize(s):
    for word in s:
        if word not in stopwords:
            clean_words.append(word)
    return clean_words

tmp2 = tmp.withColumn('final', stopCols('preprocessed'))

In [10]:
tmp2

author,book_id,category,date,description,image,ranking,title,preprocessed,final
토마스 불핀치,2855,신화,1648447114000,중학생 독후감 필독선 시리즈 제13권 토마스 불핀치의 명작 그리스 로마 신화를 작품 알고 들어가기 내용 훑어보기 작품 분석하기 작가 들여다보기 시대와 연관짓기 작품 토론하기 ...,https://bookthumb-phinf.pstatic.net/cover/000/028/00002855.jpg?type=m5,34,중학생이 보는 그리스 로마 신화,"[중학생, 독후감, 필, 독선, 시리즈, 제, 권, 토마스, 불, 핀치, 명작, 그리스, 로마, 신화, 작품, 알, 내용, 훑어보기, 작품, 분석, 작가, 시대, 연관, 작품...","[중학생, 독후감, 필, 독선, 시리즈, 권, 토마스, 불, 핀치, 명작, 그리스, 로마, 신화, 작품, 내용, 훑어보기, 작품, 분석, 작가, 시대, 연관, 작품, 토론, ..."
효성출판사 편집부,40565,가구/DIY,1648449401000,분위기기능에 따른 가구선택과 배치 주문가구 정보전국 5대 가구시장을 컬러화보 위주로 엮은 인테리어 무크지,https://bookthumb-phinf.pstatic.net/cover/000/405/00040565.jpg?type=m5,37,소가구 & 수납가구,"[분위기, 기능, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 가구, 시장, 컬러, 화보, 위주, 엮, 인테리어, 무크]","[분위기, 기능, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 가구, 시장, 컬러, 화보, 위주, 엮, 인테리어, 무크]"
중앙M&B 편집부,66769,가구/DIY,1648449401000,실내 벽에서 소품가구까지 활용 가능한 페인팅 기법 을 컬러사진과 함께 설명한 생활무크 페인팅에 필요 한 도구와 재료설명을 비롯해 색다르게 칠하는 페인팅기법리폼 및 장식 아이디...,https://bookthumb-phinf.pstatic.net/cover/000/667/00066769.jpg?type=m5,36,내 손으로 더 예쁘게 페인팅 DIY,"[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설명, 생활, 무크, 페인팅, 필요, 도구, 재료, 설명, 비롯, 색다르다, 칠하다, 페인, ...","[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설명, 생활, 무크, 페인팅, 필요, 도구, 재료, 설명, 비롯, 색다르다, 칠하다, 페인, ..."
제레미 리프킨,118837,경제/경영,null,산업 시대 초기에 석탄과 증기 기관이 새로운 경제 패러다임을 마련했듯이 이제 수소 에너지가 기존의 경제 정치 사회를 근본적으로 바꿀 것이라고 예견하는 경제서 노동의 종말 소유...,https://bookthumb-phinf.pstatic.net/cover/001/188/00118837.jpg?type=m5,27,수소 혁명,"[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 이제, 수소, 에너지, 기존, 경제, 정치, 사회, 근본, 것, 예견, 경제, 노동, 종말, 소...","[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 이제, 수소, 에너지, 기존, 경제, 정치, 근본, 예견, 경제, 노동, 종말, 소유, 종말, ..."
편집부,149663,비즈/구슬공예,1648449269000,스와롭스키 비즈로 귀엽고 깜찍한 동물들을 만든다 꿰는 방법을 쉽게 이해하도록 평면도로 그려 놓았으며 완성된 크기와 사용할 비즈 낚싯줄 길이 공정도 꿰는 방법 등을 세세하게 기...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149663.jpg?type=m5,39,비즈 캐릭터,"[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 꿰, 방법, 이해, 평면도, 완성, 크기, 사용, 비즈, 낚싯줄, 길이, 공정, 꿰, 방법, 등, 기록, 누구, 작품, ...","[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 꿰, 방법, 이해, 평면도, 완성, 크기, 사용, 비즈, 낚싯줄, 길이, 공정, 꿰, 방법, 기록, 작품]"
노경민,149664,비즈/구슬공예,1648449268000,비즈 공예 입문서이자 전문서 PART 1에서는 시작 마무리 규칙적인 기법 비즈의 기본적인 종류와 명칭 간단한 도구들에 대해 알아보며 PART 2에서는 진주 PART 3에서는 ...,https://bookthumb-phinf.pstatic.net/cover/001/496/00149664.jpg?type=m5,38,비즈 주얼리,"[비즈, 공예, 입문, 이자, 전문, 시작, 마무리, 규칙, 기법, 비즈, 기본, 종류, 명칭, 간단하다, 도구, 대해, 진주, 크리스털, 이용, 기본, 기법, 비즈, 공예,...","[비즈, 공예, 입문, 이자, 전문, 시작, 마무리, 규칙, 기법, 비즈, 기본, 종류, 명칭, 간단하다, 도구, 진주, 크리스털, 이용, 기본, 기법, 비즈, 공예, 법, ..."
편집부,150897,비즈/구슬공예,1648449270000,비즈 만드는 법 303가지를 모아 엮었다 반지 목걸이 팔찌 귀걸이 등 다양한 악세사리 제작 방법과 비즈의 종류 등에 대하여 일목요연하게 기술하고 있으며 부재료 사용법과 게재작...,https://bookthumb-phinf.pstatic.net/cover/001/508/00150897.jpg?type=m5,40,토탈 비즈 컬렉션,"[비즈, 법, 가지, 엮었다, 반지, 목걸이, 팔찌, 귀걸이, 등, 다양하다, 악세사리, 제작, 방법, 비즈, 종류, 등, 대하, 일목요연하다, 기술, 있다, 부, 재료, 용...","[비즈, 법, 엮었다, 반지, 목걸이, 팔찌, 귀걸이, 다양하다, 악세사리, 제작, 방법, 비즈, 종류, 일목요연하다, 기술, 부, 재료, 용법, 게재, 작품, 제작, 방법,..."
사메지마 다카코,150950,비즈/구슬공예,1648449268000,세상에서 단 하나뿐인 나만의 보석 만들기 로맨틱한 디자인부터 귀엽고 깜찍한 디자인까지 반지 초커 피어스 목걸이 핸드폰 줄 헤어 핀 등을 이용하여 사랑스럽고 기품있는 비즈 쥬얼...,https://bookthumb-phinf.pstatic.net/cover/001/509/00150950.jpg?type=m5,37,프린세스 비즈,"[세상, 단, 하나, 나, 보석, 만들기, 로맨틱, 디자인, 귀엽다, 깜찍하다, 디자인, 반지, 초, 피어스, 목걸이, 핸드폰, 줄, 헤어, 핀, 등, 이용, 사랑스럽다, 기...","[세상, 단, 보석, 만들기, 로맨틱, 디자인, 귀엽다, 깜찍하다, 디자인, 반지, 초, 피어스, 목걸이, 핸드폰, 헤어, 핀, 이용, 사랑스럽다, 기품, 비즈, 쥬얼리, 기..."
서울문화사 편집부,155380,비즈/구슬공예,1648449267000,3천원으로 제작하는 명품 스타일 비즈 액세서리 이 책은 비즈로 팔찌 발찌 반지 브로치 등의 액세서리를 포함해서 핸드폰줄과 키홀더 등도 제작할 수 있도록 구성되었다 종류별로 구...,https://bookthumb-phinf.pstatic.net/cover/001/553/00155380.jpg?type=m5,35,3천원으로 명품 스타일 비즈 액세서리,"[제작, 명품, 스타일, 비즈, 액세서리, 이, 책, 비즈, 팔찌, 발찌, 반지, 브로치, 등, 액세서리, 포함, 핸드폰, 줄, 키홀, 더, 등, 제작, 수, 있다, 구성, ...","[제작, 명품, 스타일, 비즈, 액세서리, 책, 비즈, 팔찌, 발찌, 반지, 브로치, 액세서리, 포함, 핸드폰, 키홀, 제작, 구성, 종류, 별로, 구별, 찾기, 해, 작품,..."
서울문화사 편집부,155381,비즈/구슬공예,1648449267000,탄생석 목걸이 · 귀고리 96가지 이 책은 비즈로 목걸이와 귀고리를 제작할 수 있도록 구성되었다 목걸이 귀고리 목걸이 귀고리 세트로 구별해 놓아 찾기 쉽게 해 놓았으며 작품...,https://bookthumb-phinf.pstatic.net/cover/001/553/0

In [16]:
tmp2.select('final')

final
"[중학생, 독후감, 필, 독선, 시리즈, 권, 토마스, 불, 핀치, 명작, 그리스, 로마, 신화, 작품, 내용, 훑어보기, 작품, 분석, 작가, 시대, 연관, 작품, 토론, ..."
"[분위기, 기능, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 가구, 시장, 컬러, 화보, 위주, 엮, 인테리어, 무크]"
"[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설명, 생활, 무크, 페인팅, 필요, 도구, 재료, 설명, 비롯, 색다르다, 칠하다, 페인, ..."
"[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 이제, 수소, 에너지, 기존, 경제, 정치, 근본, 예견, 경제, 노동, 종말, 소유, 종말, ..."
"[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 꿰, 방법, 이해, 평면도, 완성, 크기, 사용, 비즈, 낚싯줄, 길이, 공정, 꿰, 방법, 기록, 작품]"
"[비즈, 공예, 입문, 이자, 전문, 시작, 마무리, 규칙, 기법, 비즈, 기본, 종류, 명칭, 간단하다, 도구, 진주, 크리스털, 이용, 기본, 기법, 비즈, 공예, 법, ..."
"[비즈, 법, 엮었다, 반지, 목걸이, 팔찌, 귀걸이, 다양하다, 악세사리, 제작, 방법, 비즈, 종류, 일목요연하다, 기술, 부, 재료, 용법, 게재, 작품, 제작, 방법,..."
"[세상, 단, 보석, 만들기, 로맨틱, 디자인, 귀엽다, 깜찍하다, 디자인, 반지, 초, 피어스, 목걸이, 핸드폰, 헤어, 핀, 이용, 사랑스럽다, 기품, 비즈, 쥬얼리, 기..."
"[제작, 명품, 스타일, 비즈, 액세서리, 책, 비즈, 팔찌, 발찌, 반지, 브로치, 액세서리, 포함, 핸드폰, 키홀, 제작, 구성, 종류, 별로, 구별, 찾기, 해, 작품,..."
"[탄생석, 목걸이, 귀고리, 책, 비즈, 목걸이, 귀고리, 제작, 구성, 목걸이, 귀고리, 목걸이, 귀고리, 세트, 구별, 찾기, 해, 작품, 난이도, 제작, 표시, 관련, ..."


In [17]:
# tmp2.select('preprocessed').collect()

불용어 제거 전과 후의 차이 크게 없음

한국어는 따로 정해진 불용어는 없고(nltk 한국어 불용어 리스트 없음) 토큰화 단계에서 조사.접속사를 제거하면 됨

필요없는 명사/형용사 제거하고 싶을 경우 직접 불용어 리스트 정의 후 한국어 불용어 제거하면 됨

## Modeling - Word2Vec

In [11]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import Word2VecModel

# inputCol=input Column name 
w2v = Word2Vec(vectorSize=100, seed=42, inputCol="preprocessed", outputCol="model")
model = w2v.fit(tmp2)
model.setInputCol("preprocessed")
model.getVectors().show()

22/06/07 06:17:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/06/07 06:17:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


+--------+--------------------+
|    word|              vector|
+--------+--------------------+
|  호흡기|[0.01659609936177...|
|    근교|[0.05370462685823...|
|    사진|[-0.1408850848674...|
|    암시|[-0.0031537965405...|
|      스|[0.14684820175170...|
|  정재승|[0.09937392175197...|
|  파닉스|[0.25372108817100...|
|    마이|[-0.0205087121576...|
|  신격호|[0.02725019119679...|
|    권태|[0.02631604671478...|
|    당국|[0.00310591422021...|
|  십자가|[0.01388292293995...|
|    가드|[0.01179429516196...|
|    한강|[0.03636000305414...|
|  집주인|[0.03039329499006...|
|    오기|[0.05870921164751...|
|    관리|[0.04083989188075...|
|    부사|[0.00605130009353...|
|불리하다|[0.24576191604137...|
|    절박|[-0.0488759167492...|
+--------+--------------------+
only showing top 20 rows



In [19]:
print(model.getVectorSize())
print(model.getVectors().first())

100
Row(word='호흡기', vector=DenseVector([0.0166, 0.0048, -0.0289, -0.0101, 0.0031, -0.0112, 0.0257, 0.0018, 0.0156, 0.03, -0.0072, 0.0061, -0.0272, -0.0055, 0.0158, 0.0057, -0.0272, -0.0144, 0.0007, -0.0114, 0.0183, 0.0199, 0.0111, 0.005, 0.0052, 0.0045, -0.0001, 0.0075, 0.0017, -0.0214, -0.0017, -0.013, -0.0309, -0.0081, -0.0134, 0.0056, -0.0002, 0.0317, -0.0374, -0.0198, -0.0016, -0.0275, 0.025, 0.004, 0.0322, -0.0024, 0.0024, -0.0049, -0.0174, -0.0237, 0.0234, -0.0193, 0.0384, 0.0084, 0.0088, -0.0066, -0.0156, -0.0003, 0.0145, 0.0302, 0.0265, 0.0113, -0.0337, -0.0149, 0.0065, 0.0148, 0.0053, 0.0208, 0.0008, -0.0085, 0.0156, -0.0196, -0.0075, -0.0025, 0.0117, 0.0089, -0.0274, -0.0169, 0.0335, 0.0205, 0.009, -0.0209, 0.0182, -0.0449, -0.0035, 0.0057, 0.0054, 0.0177, -0.0078, 0.0039, -0.0228, -0.0033, 0.0173, 0.0034, 0.0067, -0.0262, 0.0052, 0.0027, -0.0092, 0.0251]))


#### 결과 확인

In [20]:
model

Word2VecModel: uid=Word2Vec_23ca0c50c497, numWords=10710, vectorSize=100

In [21]:
from pyspark.sql.functions import format_number as fmt

model.findSynonyms("사진", 2).select("word", fmt("similarity", 5).alias("similarity"))

word,similarity
촬영,0.75694
채색,0.70983


### 모델 저장

In [23]:
temp_path = "./"
# model metadata
w2vPath = temp_path + "/word2vec2"
w2v.save(w2vPath)

# metadata, weight in parquet form
modelPath = temp_path + "/word2vec-model2"
model.save(modelPath)

In [24]:
tmp2.select('preprocessed').collect()

[Row(preprocessed=['중학생', '독후감', '필', '독선', '시리즈', '제', '권', '토마스', '불', '핀치', '명작', '그리스', '로마', '신화', '작품', '알', '내용', '훑어보기', '작품', '분석', '작가', '시대', '연관', '작품', '토론', '독후감', '예시', '독후감', '길라잡이', '책', '전', '책', '감상', '방법', '독후감', '무엇', '의', '독후감', '제대로', '쓰기', '코너', '통해', '이해', '수', '있다', '구성']),
 Row(preprocessed=['분위기', '기능', '가구', '선택', '배치', '주문', '가구', '정보', '전국', '가구', '시장', '컬러', '화보', '위주', '엮', '인테리어', '무크']),
 Row(preprocessed=['실내', '벽', '소품', '가구', '활용', '가능하다', '페인', '팅', '기법', '컬러', '사진', '설명', '생활', '무크', '페인팅', '필요', '도구', '재료', '설명', '비롯', '색다르다', '칠하다', '페인', '법리', '폼', '및', '장식', '아이디어', '소개']),
 Row(preprocessed=['산업', '시대', '초기', '석탄', '증기', '기관', '새롭다', '경제', '패러다임', '마련', '이제', '수소', '에너지', '기존', '경제', '정치', '사회', '근본', '것', '예견', '경제', '노동', '종말', '소유', '종말', '저자', '러미', '리프킨', '최신', '작', '작가', '이', '책', '지구', '가장', '근본', '가장', '구', '수', '있다', '자원', '수소', '인간', '문명', '성하다', '세계', '경제', '권력', '구조', '재', '편하다', '것', '말', '있다', '모든', '사람', '소비자', '동시', '잠재', '에너

In [12]:
pandas_wv = model.getVectors().toPandas()

In [14]:
pandas_wv.head(5)

,word,vector
0,호흡기,"[0.016596099361777306, 0.00478344364091754, -0..."
1,근교,"[0.053704626858234406, -0.022303609177470207, ..."
2,사진,"[-0.14088508486747742, 0.1455814242362976, -0...."
3,암시,"[-0.0031537965405732393, 0.007566554006189108,..."
4,스,"[0.14684820175170898, 0.1128878965973854, 0.29..."


In [13]:
a = pandas_wv.loc[pandas_wv['word']=='근교'].vector
a

1    [0.053704626858234406, -0.022303609177470207, ...
Name: vector, dtype: object

In [27]:
pandas_wv["word"]

0         호흡기
1          근교
2          사진
3          암시
4           스
         ... 
10705       십
10706      분리
10707       칼
10708      날로
10709    씩씩하다
Name: word, Length: 10710, dtype: object

In [15]:
tmp_pd = tmp2.toPandas(); tmp_pd

,author,book_id,category,date,description,image,ranking,title,preprocessed,final
0,토마스 불핀치,2855,신화,1.648447e+12,중학생 독후감 필독선 시리즈 제13권 토마스 불핀치의 명작 그리스 로마 신화를 작품...,https://bookthumb-phinf.pstatic.net/cover/000/...,34,중학생이 보는 그리스 로마 신화,"[중학생, 독후감, 필, 독선, 시리즈, 제, 권, 토마스, 불, 핀치, 명작, 그...","[중학생, 독후감, 필, 독선, 시리즈, 권, 토마스, 불, 핀치, 명작, 그리스,..."
1,효성출판사 편집부,40565,가구/DIY,1.648449e+12,분위기기능에 따른 가구선택과 배치 주문가구 정보전국 5대 가구시장을 컬러화보 위주로...,https://bookthumb-phinf.pstatic.net/cover/000/...,37,소가구 & 수납가구,"[분위기, 기능, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 가구, 시장, ...","[분위기, 기능, 가구, 선택, 배치, 주문, 가구, 정보, 전국, 가구, 시장, ..."
2,중앙M&B 편집부,66769,가구/DIY,1.648449e+12,실내 벽에서 소품가구까지 활용 가능한 페인팅 기법 을 컬러사진과 함께 설명한 생활무...,https://bookthumb-phinf.pstatic.net/cover/000/...,36,내 손으로 더 예쁘게 페인팅 DIY,"[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설...","[실내, 벽, 소품, 가구, 활용, 가능하다, 페인, 팅, 기법, 컬러, 사진, 설..."
3,제레미 리프킨,118837,경제/경영,NaN,산업 시대 초기에 석탄과 증기 기관이 새로운 경제 패러다임을 마련했듯이 이제 수소 ...,https://bookthumb-phinf.pstatic.net/cover/001/...,27,수소 혁명,"[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 이제...","[산업, 시대, 초기, 석탄, 증기, 기관, 새롭다, 경제, 패러다임, 마련, 이제..."
4,편집부,149663,비즈/구슬공예,1.648449e+12,스와롭스키 비즈로 귀엽고 깜찍한 동물들을 만든다 꿰는 방법을 쉽게 이해하도록 평면도...,https://bookthumb-phinf.pstatic.net/cover/001/...,39,비즈 캐릭터,"[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 꿰, 방법, 이해, 평면도, ...","[스, 롭스, 키, 비즈, 귀엽다, 깜찍하다, 동물, 꿰, 방법, 이해, 평면도, ..."
...,...,...,...,...,...,...,...,...,...,...
4786,안송이,22314367,시/에세이,1.648780e+12,대한민국 주부의 유쾌 발랄 종이접기 에세이 핸드메이드 글로벌 마켓 엣시에서 전 세계...,https://bookthumb-phinf.pstatic.net/cover/223/...,9,종이접기처럼 살고 싶어서,"[대한민국, 주부, 유쾌, 발랄, 종이접기, 에세이, 핸드, 메이드, 글로벌, 마켓...","[대한민국, 주부, 유쾌, 발랄, 종이접기, 에세이, 핸드, 메이드, 글로벌, 마켓..."
4787,조지 고든 바이런,22314371,시/에세이,1.648780e+12,‘질풍노도’의 시대에 낭만주의를 열다\n\n윌리엄 워즈워스 퍼시 셸리 존 키츠 등으...,https://bookthumb-phinf.pstatic.net/cover/223/...,22,차일드 해럴드의 순례,"[질풍, 노도, 의, 시대, 낭만, 주의, 윌리엄, 워즈워스, 퍼시, 셸리, 존, ...","[질풍, 노도, 시대, 낭만, 주의, 윌리엄, 워즈워스, 퍼시, 셸리, 존, 키츠,..."
4788,박신후,22314401,경제/경영,1.648780e+12,동그랗고 큰 얼굴에 길쭉한 눈 낮은 코에 씩 올라간 입꼬리 볼에 박힌 주근깨까지 보...,https://bookthumb-phinf.pstatic.net/cover/223/...,11,"행복을 파는 브랜드, 오롤리데이","[동그랗다, 얼굴, 길쭉하다, 눈, 낮다, 코, 입, 꼬리, 볼, 주근깨, 보기, ...","[동그랗다, 얼굴, 길쭉하다, 눈, 낮다, 코, 입, 꼬리, 볼, 주근깨, 보기, ..."
4789,해루,22314474,소설,1.648780e+12,따뜻한 바람이 부는 이야기마을과 얼어붙기만 하는 냉기가 가득한 얼음마을아빠의 아빠의...,https://bookthumb-phinf.pstatic.net/cover/223/...,26,황금알과 이야기나무 술술 (컬러판),"[따뜻하다, 바람, 이야기, 마을, 냉기, 가득하다, 얼음, 마을, 아빠, 아빠, ...","[따뜻하다, 바람, 이야기, 마을, 냉기, 가득하다, 얼음, 마을, 아빠, 아빠, ..."


In [43]:
def get_document_vectors(document_list):
    document_list = document_list.replace("[", "").replace("]", "").replace(" ", "").split(",")
    doc2vec = None; count = 0
    for w in document_list:
        if w in w2v_words.tolist():
            count += 1
            # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
            if doc2vec is None:
                doc2vec = pandas_wv.loc[pandas_wv['word']== w ].vector
            else:
                doc2vec = doc2vec + pandas_wv.loc[pandas_wv['word']== w ].vector

    if doc2vec is not None:
        doc2vec = doc2vec / count
    return doc2vec

w2v_words = pandas_wv['word']
tmp_pd['mean_vec'] = tmp_pd['final'].apply(get_document_vectors)
tmp_pd.final

TypeError: object of type 'NoneType' has no len()

In [44]:
tmp_pd['mean_vec']

0       36
1       16
2       24
3       51
4       20
        ..
4786    47
4787    89
4788    79
4789    21
4790    58
Name: mean_vec, Length: 4791, dtype: int64

In [58]:
vecCols = udf(lambda x : get_document_vectors(x), DoubleType())
spark.udf.register("vecCols", vecCols)

def get_document_vectors(document_list):
    doc2vec = None; count = 0
    for w in document_list:
        if w in w2v_words:
            count += 1
            # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
            if doc2vec is None:
                doc2vec = w_v_pd.loc[w_v_pd['word']== w ].vector
            else:
                doc2vec = doc2vec + w_v_pd.loc[w_v_pd['word']== w ].vector

    if doc2vec is not None:
        # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
        doc2vec = doc2vec / count

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return doc2vec 

w2v_words = [row["word"] for row in model.getVectors().collect()]
vectorize_book = tmp2.withColumn('description_to_vec', vecCols('preprocessed'))


22/06/05 07:09:00 WARN SimpleFunctionRegistry: The function veccols replaced a previously registered function.


In [32]:
len(w2v_words)

10710

In [59]:
vectorize_book

22/06/05 07:09:25 ERROR Executor: Exception in task 0.0 in stage 44.0 (TID 177)]
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCo

Py4JJavaError: An error occurred while calling o596.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 177) (57be026d115b executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$1

22/06/05 07:09:51 ERROR Executor: Exception in task 0.0 in stage 45.0 (TID 178)]
net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCo

Py4JJavaError: An error occurred while calling o596.getRowsToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 178) (57be026d115b executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:213)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:490)
	at scala.collection.Iterator$$a